# Analyze and Extract Savannah Polygon

I'm using the shape file found on the [sagis](https://data-sagis.opendata.arcgis.com/datasets/municipality?geometry=-82.512%2C31.772%2C-79.719%2C32.180) portal.  The file has been downloaded to the data directory.

For the extract from the shape file to a geojson file I'm using a [recipe](https://gist.github.com/drmalex07/5a54fc4f1db06a66679e) I found after a brief consultation with google.

Steps in this notebook are quite straight forward:

>1. Read the file with geopandas and explore the geodataframe.
>2. Quick visual display using ipyleaflet.
>3. Use the recipe to save the geometry from the geodataframe to a geojson format.


In [ ]:
import geopandas as gpd

from ipyleaflet import (Map, GeoJSON,
                        GeoData, LayersControl,
                        basemaps, basemap_to_tiles)

from ipywidgets import Layout 
import json
import geojson
import ast

from shapely.wkt import loads
from shapely.geometry import mapping

## Step 1 - read and explore

Not a whole lot to explain on this.  I just wanted to understand what's in it.  Used the square miles to determine that this is the one.

In [ ]:
municipality_gdf = gpd.read_file('../data/savannah/Municipality.shp')

In [ ]:
municipality_gdf.columns

In [ ]:
municipality_gdf[['NAME', 'SQ_MILES', 'geometry']]

In [ ]:
municipality_gdf.iloc[3]

In [ ]:
savannah_record = municipality_gdf.iloc[3]
savannah = savannah_record.geometry

In [ ]:
savannah_polygons = list(savannah)

In [ ]:
len(savannah_polygons)

In [ ]:
savannah_polygons

## Step 2 - display

This is a standard idiom for me.  

1. I use the savannah geometry to get the centroid for map positioning
2. Create a geodataframe specific to savannah
3. Use that dataframe to create overlay for the map.

In [ ]:
x = savannah.centroid.x
y = savannah.centroid.y

In [ ]:
savannah_gdf = municipality_gdf.query(f"NAME == 'SAVANNAH'").reset_index()

In [ ]:
savannah_gdf

In [ ]:
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True


map_display = Map(center=(y, x), zoom=12,
                  layers=[imagery, osm],
                  layout=Layout(height="900px"),
                  scroll_wheel_zoom=True)

map_display.add_control(LayersControl())
map_display

In [ ]:
municipal_polygons = GeoData(geo_dataframe = savannah_gdf,
                             style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                             hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                             name = 'savannah')

map_display += municipal_polygons


## Step 3 - generate geojson

Result of this section is the output file for use in the treepedia project.

Easy to change if other municipalities are important later.

In [ ]:
savannah_geo = savannah_gdf.geometry

In [ ]:
geojson_string = geojson.dumps(mapping(loads(savannah_geo.iloc[0].wkt)))

In [ ]:
geojson_dict = ast.literal_eval(geojson_string)

In [ ]:
with open('savannah.json', 'w') as geojson_file:
    geojson_file.write(json.dumps(geojson_dict))

In [ ]:
savannah_gdf.iloc[0].geometry